In [ ]:
api_key = "apikey"   # api key of pinecone

In [ ]:
from langchain_community.retrievers import PineconeHybridSearchRetriever 

In [ ]:
import os
from pinecone import Pinecone,ServerlessSpec
index_name = "pinecone-hybrid-search-tutorial-for-beginners"

# initializing the Pinecone client
pc = Pinecone(api_key=api_key)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,   # dimension of dense vector
        metric='dotproduct',  # sparse values supported for dotproduct
        spec=ServerlessSpec(cloud='aws',region="us-east-1")
    )

In [ ]:
index=pc.Index(index_name)
index

## Vector embedding and sparse matrix

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_huggingface import HuggingFaceEmbeddings   # open source embedding technique 
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

In [ ]:
from pinecone_text.sparse import BM25Encoder
bm25_encoder = BM25Encoder().default()   # TFIDF
bm25_encoder

In [ ]:
sentences = [
    "Pakistan won the 2017 ICC Champions Trophy."

    "Australia and India have dominated cricket in recent years."

    "The ICC Champions Trophy will return in 2025."
]
# tfidf values on these sentences
bm25_encoder.fit(sentences)

# store the value in json file
bm25_encoder.dump("bm25_values.json")

# load BM25Encoder Object
bm25_encoder=BM25Encoder().load("bm25_values.json")

In [ ]:
retriver=PineconeHybridSearchRetriever(embeddings=embeddings,parse_encoder=bm25_encoder,index=index)

In [ ]:
retriver

In [ ]:
retriver.add_texts(
    sentences = [
    "Pakistan won the 2017 ICC Champions Trophy."

    "Australia and India have dominated cricket in recent years."

     "The ICC Champions Trophy will return in 2025."
]
)

In [ ]:
retriver.invoke('who won the ICC Champions Tropy in 2017')